In [1]:
# Data
import os

# Set data path
data_path = '/kaggle/input/sarscope-unveiling-the-maritime-landscape/SARscope'

# Check 
print("Dataset structure:", os.listdir(data_path))


Dataset structure: ['README.roboflow.txt', 'valid', 'test', 'train']
